This notebook generates a Hovmoller plot of temperature vs. depth for the abrupt experiments.

In [1]:
import cosima_cookbook as cc
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean as cm
import IPython.display

from dask.distributed import Client

%matplotlib inline

In [2]:
client = Client()
display(client)

Client Scheduler: tcp://127.0.0.1:41199 Dashboard: /proxy/8787/status,Cluster Workers: 3 Cores: 6 Memory: 25.77 GB


In [3]:
### Generate new database

db = '/scratch/e14/mp2135/access-om2/archive/databases/1deg_jra55_ryf_experiments_abrupt_spinup500.db'

session = cc.database.create_session(db)

In [4]:
cc.querying.get_experiments(session)

,experiment,ncfiles
0,1deg_jra55_ryf_cont,150429
1,output111,2096
2,output112,2096
3,output113,2096
4,output114,2096
5,output115,2096
6,output116,2096
7,output117,2096
8,output118,2096
9,output119,2096


In [5]:
# Control

control_expt = '1deg_jra55_ryf_cont'
control = cc.querying.get_variables(session, experiment = control_expt)

temp_control = cc.querying.getvar(control_expt, 'temp', session, frequency = '1 monthly').chunk(chunks='auto')
 # [:-1] to remove 2200-01-01:
GAT_control = cc.querying.getvar(control_expt, 'temp_global_ave', session)[:-1].chunk(chunks=3650).load().groupby('time.year').mean('time')
SST_control = cc.querying.getvar(control_expt, 'temp_surface_ave', session)[:-1].chunk(chunks=3650).load().groupby('time.year').mean('time')

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

In [ ]:
# Cool

cool_expt = '1deg_jra55_ryf_rcp45cool'
cool = cc.querying.get_variables(session, experiment = cool_expt)

temp_cool = cc.querying.getvar(cool_expt, 'temp', session, frequency = '1 monthly').chunk(chunks='auto')
GAT_cool = cc.querying.getvar(cool_expt, 'temp_global_ave', session)[:-1].chunk(chunks=3650).load().groupby('time.year').mean('time')
SST_cool = cc.querying.getvar(cool_expt, 'temp_surface_ave', session)[:-1].chunk(chunks=3650).load().groupby('time.year').mean('time')

GAT_cool_anom = GAT_cool - GAT_control
SST_cool_anom = SST_cool - SST_control

In [ ]:
# Warm

warm_expt = '1deg_jra55_ryf_rcp45warm'
warm = cc.querying.get_variables(session, experiment = warm_expt)

temp_warm = cc.querying.getvar(warm_expt, 'temp', session, frequency = '1 monthly').chunk(chunks='auto')
GAT_warm = cc.querying.getvar(warm_expt, 'temp_global_ave', session)[:-1].chunk(chunks=3650).load().groupby('time.year').mean('time')
SST_warm = cc.querying.getvar(warm_expt, 'temp_surface_ave', session)[:-1].chunk(chunks=3650).load().groupby('time.year').mean('time')

GAT_warm_anom = GAT_warm - GAT_control
SST_warm_anom = SST_warm - SST_control

Next, we load cell area (denoted $a(x,y,z)$) from the ocean_grid file, construct a mask from the temperature file and make a profile of the total ocean area as a function of depth, $A$, namely
$$ A(z) = \sum_x \sum_y a(x,y,z)$$
*Note that this mask ignores the presence of partial cells, for now*.

In [ ]:
cc.querying.get_variables(session, control_expt)[cc.querying.get_variables(session, control_expt)['name'].str.lower().str.match('area')]

In [ ]:
area_t = cc.querying.getvar(control_expt, 'area_t', session, n=1)#.mean('time')
mask = temp_control.isel(time=0).copy()
mask = mask / mask                 ## This seems pretty dodgy to me, but it works!
area = mask * area_t
area_sum = area.sum('xt_ocean').sum('yt_ocean')
IPython.display.clear_output()

Now, the mean temperature at each time level can then be computed as 
$$T(z,t) = \frac{\sum_x \sum_y a(x,y,z) \theta(x,y,z,t)}{A(z)}$$
where $T$ is the average temperature and $\theta$ the potential temperature.

In [ ]:
var = area_t * (temp_cool - temp_control)
temp_hov = var.sum('xt_ocean').sum('yt_ocean').chunk(chunks='auto').compute()
temp_hov_cool = temp_hov / area_sum

var = area_t * (temp_warm - temp_control)
temp_hov = var.sum('xt_ocean').sum('yt_ocean').chunk(chunks='auto').compute()
temp_hov_warm = temp_hov / area_sum

We can use xarray for a quick and dirty plot of this data:

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize = (21,9))
p1 = temp_hov_cool[:,:33].T.plot.contourf(ax=ax[0], levels=45,yincrease=False)
p2 = temp_hov_warm[:,:33].T.plot.contourf(ax=ax[1], levels=45,yincrease=False, add_colorbar=False)
cb = plt.colorbar(p1,ax=ax[1],orientation='vertical')